In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!pip install datasets
!huggingface-cli login

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 25.3 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://hugg

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
import os
from tqdm import tqdm
import traceback

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load MBPP dataset
ds = load_dataset("google-research-datasets/mbpp", "full")
test_data = ds["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

...

prompt-00000-of-00001.parquet:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

# Debugging, testing on single row

In [ ]:
# For example, select a single sample from the MBPP test dataset.
# (Adjust the index as needed.)
sample = test_data[1]
prompt = sample["text"]       # The task prompt (or question)
reference = sample["code"]    # The reference solution/code
test_list = sample.get("test_list", [])  # List of tests (if present)

# Optional: Print these out for confirmation.
print("=== Defined Variables ===")
print("Task ID:", sample["task_id"])
print("Prompt:\n", prompt)
print("\nReference Code:\n", reference)
print("\nTest List:", test_list)

=== Defined Variables ===
Task ID: 12
Prompt:
 Write a function to sort a given matrix in ascending order according to the sum of its rows.

Reference Code:
 def sort_matrix(M):
    result = sorted(M, key=sum)
    return result

Test List: ['assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]', 'assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]', 'assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]']


# Improved Iteration Testing

# With Pass@k Interpretation AND SELF CONSISTENCY

In [9]:
import re
import traceback
import os
import json
import time
import math
import torch
import ast
import signal
from contextlib import contextmanager
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
from functools import lru_cache

@contextmanager
def time_limit(seconds):
    """
    Raises TimeoutError if the code block inside the context
    takes more than `seconds` to execute.
    """
    def signal_handler(signum, frame):
        raise TimeoutError("Code execution timed out.")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


def extract_code(text, func_name=None):
    """
    Unified code extraction function that handles multiple extraction strategies:
    1. Extract all triple-backtick code blocks
    2. If a function name is provided, extract just that function
    3. Fall back to searching for function definition in raw text

    Returns the first valid code block that can be parsed, or raises ValueError.
    """
    # Try to extract from triple backticks first
    pattern = r"```(.*?)```"
    blocks = re.findall(pattern, text, flags=re.DOTALL)

    valid_blocks = []
    for block in blocks:
        candidate = block.strip()

        # Handle language tags
        lines = candidate.split("\n", 1)
        if len(lines) == 2:
            first_line = lines[0].lower().strip()
            if first_line in ["python", "python3", "bash", "shell", "sql", "sh"]:
                candidate = lines[1].strip()

        # Skip empty blocks
        if not candidate or candidate.lower() in ["python", "python3"]:
            continue

        # Try to parse
        try:
            ast.parse(candidate)
            valid_blocks.append(candidate)
        except SyntaxError:
            pass

    # If we found any valid blocks, return the first one
    if valid_blocks:
        return valid_blocks[0]

    # If we have a function name, try to extract just that function
    if func_name:
        # Try from blocks first
        for block in blocks:
            try:
                extracted = extract_function(block, func_name)
                return extracted
            except ValueError:
                pass

        # Try from raw text
        try:
            return extract_function(text, func_name)
        except ValueError:
            pass

    # Nothing worked
    raise ValueError("No valid code block found and function extraction failed.")


def extract_function(text, func_name):
    """
    Extract just the function definition and body for a given function name.
    """
    pattern = rf"(\s*)def\s+{re.escape(func_name)}\s*\(.*?\):"
    lines = text.splitlines()
    func_start_idx = None
    base_indent = None

    for idx, line in enumerate(lines):
        m = re.match(pattern, line)
        if m:
            func_start_idx = idx
            base_indent = len(m.group(1).replace('\t', '    '))
            break

    if func_start_idx is None:
        raise ValueError(f"Could not find 'def {func_name}(...):' in text.")

    # Collect function body
    collected = [lines[func_start_idx]]
    for i in range(func_start_idx + 1, len(lines)):
        line = lines[i]
        # Check for next top-level definition
        next_def_m = re.match(r"^(\s*)(?:def|class)\s+\w+", line)
        if next_def_m:
            next_indent = len(next_def_m.group(1).replace('\t', '    '))
            if next_indent <= base_indent:
                break

        # Check indentation
        if line.strip() == '' or len(line) - len(line.lstrip(' ')) > base_indent:
            collected.append(line)
        else:
            break

    extracted = "\n".join(collected)

    # Validate
    try:
        ast.parse(extracted)
        return extracted
    except SyntaxError as e:
        raise ValueError(f"Extracted function body not parseable: {e}")


def extract_expected_signature(text):
    """
    Extracts the function name and parameters from a Python function signature.
    """
    pattern = r"def\s+(\w+)\s*\((.*?)\):"
    match = re.search(pattern, text)
    if match:
        return match.group(1), match.group(2)
    return None, None


@lru_cache(maxsize=1024)
def tokenize_code(code):
    """
    Tokenize code for similarity comparison, with caching.
    """
    # Replace common punctuation with spaces before splitting
    for char in '(){}[],;:.':
        code = code.replace(char, ' ' + char + ' ')
    return tuple(token for token in code.split() if token.strip())


def calculate_similarity(code1, code2):
    """
    Optimized similarity calculation combining token-based and edit distance.
    """
    # Get tokens with caching
    tokens1 = set(tokenize_code(code1))
    tokens2 = set(tokenize_code(code2))

    # Jaccard similarity for tokens
    if not tokens1 and not tokens2:
        return 1.0
    if not tokens1 or not tokens2:
        return 0.0

    intersection = tokens1.intersection(tokens2)
    union = tokens1.union(tokens2)
    token_sim = len(intersection) / len(union)

    # Simple ratio of matching tokens as approximation for edit distance
    # This avoids the expensive Levenshtein calculation
    smaller_set = min(len(tokens1), len(tokens2))
    larger_set = max(len(tokens1), len(tokens2))

    if larger_set == 0:
        ratio_sim = 1.0
    else:
        ratio_sim = smaller_set / larger_set

    # Weight token-based similarity higher
    return 0.7 * token_sim + 0.3 * ratio_sim


def select_most_consistent(responses):
    """
    Optimized selection of most consistent response using memoization.
    """
    # Extract valid code once
    valid_code_pairs = []

    for idx, response in enumerate(responses):
        try:
            code = extract_code(response)
            valid_code_pairs.append((idx, code))
        except ValueError:
            continue

    # Handle edge cases
    if len(valid_code_pairs) <= 1:
        return responses[0] if responses else ""

    # Calculate similarity matrix only for valid codes
    n = len(valid_code_pairs)
    similarity_cache = {}
    avg_similarities = [0.0] * n

    # Calculate all similarities at once to avoid redundant calculations
    for i in range(n):
        total_sim = 0.0
        idx_i, code_i = valid_code_pairs[i]

        for j in range(n):
            if i == j:
                continue

            idx_j, code_j = valid_code_pairs[j]

            # Use cached similarity if available
            pair_key = (min(idx_i, idx_j), max(idx_i, idx_j))
            if pair_key not in similarity_cache:
                similarity_cache[pair_key] = calculate_similarity(code_i, code_j)

            sim = similarity_cache[pair_key]
            total_sim += sim

        avg_similarities[i] = total_sim / (n - 1)

    # Find the most consistent code and return its original response
    most_consistent_idx = avg_similarities.index(max(avg_similarities))
    original_response_idx = valid_code_pairs[most_consistent_idx][0]

    return responses[original_response_idx]


def run_tests_on_code(code, expected_func_name, test_list):
    """
    Execute the code and run tests with unified error handling.
    """
    local_namespace = {}
    test_details = []

    # Parse check
    try:
        ast.parse(code)
    except SyntaxError as e:
        error_msg = f"Code syntax invalid: {e}"
        return False, [False]*len(test_list), error_msg, []

    # Execute with time limit
    try:
        with time_limit(5):
            exec(code, {}, local_namespace)
    except TimeoutError as e:
        error_msg = f"Code execution timeout: {e}"
        return False, [False]*len(test_list), error_msg, []
    except Exception as e:
        error_msg = f"Code execution error: {e}\n{traceback.format_exc()}"
        return False, [False]*len(test_list), error_msg, []

    # Check for function
    if expected_func_name and expected_func_name not in local_namespace:
        # Try to find a callable with similar name
        callables = [(n, obj) for n, obj in local_namespace.items() if callable(obj)]
        if len(callables) == 1:
            candidate_name, candidate_obj = callables[0]
            local_namespace[expected_func_name] = candidate_obj
        else:
            for n, obj in callables:
                if expected_func_name.lower() in n.lower() or n.lower() in expected_func_name.lower():
                    local_namespace[expected_func_name] = obj
                    break

    # Run all tests at once first to avoid individual timeouts
    all_tests_code = "\n".join(test_list)
    try:
        with time_limit(10):  # Longer timeout for all tests
            exec(all_tests_code, {}, local_namespace)
        # If we get here, all tests passed
        return True, [True]*len(test_list), "", [{"test": test, "passed": True} for test in test_list]
    except Exception:
        # Some test failed, evaluate individually
        pass

    # Evaluate each test individually
    test_pass_list = []
    for test_code in test_list:
        info = {
            "test": test_code,
            "passed": False,
            "expected": None,
            "actual": None,
            "error": None
        }

        # Check if it's an equality assertion
        m = re.match(r'assert\s+(.+?)\s*==\s*(.+)', test_code)
        if m:
            lhs_expr = m.group(1).strip()
            rhs_expr = m.group(2).strip()
            try:
                with time_limit(5):
                    lhs_val = eval(lhs_expr, {}, local_namespace)
                    rhs_val = eval(rhs_expr, {}, local_namespace)
                info["actual"] = lhs_val
                info["expected"] = rhs_val
                if lhs_val == rhs_val:
                    info["passed"] = True
                else:
                    info["error"] = f"AssertionError: {lhs_expr} = {lhs_val} != {rhs_expr} = {rhs_val}"
            except TimeoutError:
                info["error"] = "Test execution timed out."
            except Exception as e:
                info["error"] = f"Error evaluating expressions: {e}"
        else:
            # General assertion
            try:
                with time_limit(5):
                    exec(test_code, {}, local_namespace)
                info["passed"] = True
            except TimeoutError:
                info["error"] = "Test execution timed out."
            except AssertionError as e:
                info["error"] = f"AssertionError: {str(e)}"
            except Exception as e:
                info["error"] = f"Exception during test: {e}"

        test_pass_list.append(info["passed"])
        test_details.append(info)

    all_passed = all(test_pass_list)
    return all_passed, test_pass_list, "", test_details


def batch_generate(model, tokenizer, prompts, generation_config, device, batch_size=4):
    """
    Generate responses in batches for better GPU utilization.
    """
    all_responses = []

    # Process in batches
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]

        # For smaller batches, we might not need padding
        if len(batch_prompts) == 1:
            # Single prompt doesn't need padding
            inputs = tokenizer(batch_prompts[0], return_tensors="pt").to(device)

            with torch.no_grad():
                output_ids = model.generate(
                    **inputs,
                    **generation_config,
                    num_return_sequences=1
                )

            response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            all_responses.append(response)
        else:
            # Multiple prompts need padding
            batch_inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True).to(device)

            with torch.no_grad():
                output_ids = model.generate(
                    **batch_inputs,
                    **generation_config,
                    num_return_sequences=1
                )

            batch_responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            all_responses.extend(batch_responses)

    return all_responses


# Define prompts and configs once
system_prompt = (
    "You are a Python code generator.\n"
    "Return exactly one code block in triple backticks.\n"
    "No explanation, no extra text, just the function implementation.\n"
    "DO NOT EVER WRITE CODE PROMPTING FOR AN USER INPUT (LIKE DO NOT USE 'input()'), read in information from the function parameters.\n"
    "Inside that code block, define the function as requested.\n"
    "Do not look to test the implementation.\n"
    "All other text will be discarded."
)


def main():
    # Setup model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
    # Set padding token to be the same as EOS token
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Setup generation config once
    generation_config = {
        "max_new_tokens": 512,
        "temperature": 0.8,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }

    # Load dataset
    ds = load_dataset("google-research-datasets/mbpp", "full")
    test_data = ds["test"]

    # Setup output directory
    output_dir = "/content/drive/MyDrive/503/Llama_sc"
    os.makedirs(output_dir, exist_ok=True)

    # Tracking variables
    results = []
    pass_at_k_counts = {1: [], 5: [], 10: []}
    tpr_list = []
    total_gen_time = 0.0
    total_tokens_generated = 0

    # Configuration
    num_samples = 10  # Number of final samples per task
    sc_samples = 3    # Number of self-consistency samples per final sample
    batch_size = 4    # Batch size for model inference

    # Memory tracking
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        mem_before = torch.cuda.memory_allocated(device)
    else:
        mem_before = None

    # Process test samples
    test_samples = test_data.select(range(10))

    for idx, sample in enumerate(tqdm(test_samples, desc="Processing test samples")):
        prompt = sample["text"]
        reference = sample["code"]
        test_list = sample.get("test_list", [])

        # Extract function signature
        expected_func_name, expected_params = extract_expected_signature(reference)
        expected_signature_str = f"def {expected_func_name}({expected_params}):" if expected_func_name else ""

        # Build user prompt
        user_prompt = (
            f"Task:\n{prompt}\n"
            f"Required function signature:\n{expected_signature_str}\n\n"
            "Remember: only valid Python code in triple backticks."
        )

        # Combine system and user prompts
        full_prompt = f"{system_prompt}\n\nUser request:\n{user_prompt}"

        # Create all prompts for batch generation
        all_prompts = [full_prompt] * (num_samples * sc_samples)

        # Start timing
        start_time = time.time()

        # Generate all responses in batches
        all_responses = batch_generate(
            model, tokenizer, all_prompts, generation_config, device, batch_size
        )

        # Group responses for self-consistency
        sc_predictions = []
        for i in range(0, len(all_responses), sc_samples):
            group = all_responses[i:i+sc_samples]
            selected = select_most_consistent(group)
            sc_predictions.append(selected)

        # Calculate generation time
        gen_time = time.time() - start_time
        total_gen_time += gen_time

        # Count tokens
        total_tokens_generated += sum(len(tokenizer.tokenize(pred)) for pred in sc_predictions)

        # Process predictions and run tests
        correct_flags = []
        tpr_scores = []
        prediction_details = []

        for pred in sc_predictions:
            try:
                # Extract code once
                code = extract_code(pred, expected_func_name)

                # Run tests on the extracted code
                all_passed, pass_list, load_error, test_detail = run_tests_on_code(
                    code, expected_func_name, test_list
                )
            except ValueError as e:
                # Code extraction failed
                code = ""
                all_passed = False
                pass_list = [False] * len(test_list)
                load_error = str(e)
                test_detail = []

            correct_flags.append(all_passed)
            tpr = sum(pass_list) / len(pass_list) if pass_list else 0.0
            tpr_scores.append(tpr)

            prediction_details.append({
                "prediction": pred,
                "extracted_code": code,
                "all_passed": all_passed,
                "pass_list": pass_list,
                "test_details": test_detail,
                "load_error": load_error
            })

        # Calculate Pass@k metrics
        c = sum(correct_flags)
        n = len(sc_predictions)

        task_pass_at_k = {}
        for k in [1, 5, 10]:
            if n < k:
                task_pass_at_k[k] = 1.0 if c > 0 else 0.0
            else:
                try:
                    comb_n = math.comb(n, k)
                    comb_n_minus_c = math.comb(n - c, k) if (n - c) >= k else 0
                    pass_at_k_value = 1 - comb_n_minus_c / comb_n
                except Exception:
                    pass_at_k_value = 0.0
                task_pass_at_k[k] = pass_at_k_value

        # Find best prediction
        best_tpr = max(tpr_scores) if tpr_scores else 0.0
        tpr_list.append(best_tpr)
        for k in [1, 5, 10]:
            pass_at_k_counts[k].append(task_pass_at_k[k])

        best_idx = tpr_scores.index(best_tpr) if tpr_scores else 0
        best_pred = prediction_details[best_idx] if prediction_details else {}

        # Add to results
        task_result = {
            "task_id": sample["task_id"],
            "prompt": prompt,
            "reference": reference,
            "expected_signature": expected_signature_str,
            "best_prediction": best_pred.get("prediction", ""),
            "all_tests_passed": int(best_pred.get("all_passed", False)),
            "individual_test_results": best_pred.get("pass_list", []),
            "test_details": json.dumps(best_pred.get("test_details", []), indent=2),
            "load_error": best_pred.get("load_error", ""),
            "correct_count": c,
            "tpr": best_tpr,
            "pass@1": task_pass_at_k[1],
            "pass@5": task_pass_at_k[5],
            "pass@10": task_pass_at_k[10],
            "generation_time": gen_time,
            "num_tokens": sum(len(tokenizer.tokenize(pred)) for pred in sc_predictions),
        }

        results.append(task_result)

        # Save intermediate results efficiently (append to file)
        result_path = os.path.join(output_dir, "intermediate_results.jsonl")
        with open(result_path, "a") as f:
            f.write(json.dumps(task_result) + "\n")

        # Clean up to reduce memory usage
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # Calculate final metrics
    overall_pass_at_1 = sum(pass_at_k_counts[1]) / len(pass_at_k_counts[1]) if pass_at_k_counts[1] else 0.0
    overall_pass_at_5 = sum(pass_at_k_counts[5]) / len(pass_at_k_counts[5]) if pass_at_k_counts[5] else 0.0
    overall_pass_at_10 = sum(pass_at_k_counts[10]) / len(pass_at_k_counts[10]) if pass_at_k_counts[10] else 0.0
    atpr = sum(tpr_list) / len(tpr_list) if tpr_list else 0.0
    throughput = total_tokens_generated / total_gen_time if total_gen_time > 0 else 0.0

    # Memory usage
    if torch.cuda.is_available():
        mem_after = torch.cuda.memory_allocated(device)
        memory_overhead = mem_after - mem_before if mem_before is not None else None
    else:
        memory_overhead = None

    # Save final results
    df_final = pd.DataFrame(results)
    final_csv_path = os.path.join(output_dir, f"results_final_{len(test_samples)}.csv")
    df_final.to_csv(final_csv_path, index=False)

    # Save summary
    summary = {
        "pass@1": overall_pass_at_1,
        "pass@5": overall_pass_at_5,
        "pass@10": overall_pass_at_10,
        "atpr": atpr,
        "throughput": throughput,
        "memory_overhead": memory_overhead,
        "total_generation_time": total_gen_time,
        "total_tokens": total_tokens_generated
    }

    summary_path = os.path.join(output_dir, f"summary_{len(test_samples)}.json")
    with open(summary_path, "w") as f:
        json.dump(summary, f, indent=2)

    # Print metrics
    print("=== Metrics Summary ===")
    print(f"Pass@1: {overall_pass_at_1:.2f}")
    print(f"Pass@5: {overall_pass_at_5:.2f}")
    print(f"Pass@10: {overall_pass_at_10:.2f}")
    print(f"Average Test Pass Rate (ATPR): {atpr:.2f}")
    print(f"Throughput (tokens/sec): {throughput:.2f}")
    if memory_overhead is not None:
        print(f"Memory Overhead (bytes): {memory_overhead}")


if __name__ == "__main__":
    main()

Processing test samples:  50%|█████     | 5/10 [07:16<07:06, 85.39s/it]

helloworld
abcefg


Processing test samples: 100%|██████████| 10/10 [14:22<00:00, 86.28s/it]

=== Metrics Summary ===
Pass@1: 0.19
Pass@5: 0.38
Pass@10: 0.40
Average Test Pass Rate (ATPR): 0.53
Throughput (tokens/sec): 58.33
Memory Overhead (bytes): 0
